<a href="https://colab.research.google.com/github/calvindoro/pyschic-dream/blob/main/CLTV_CreditScoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WHY

CLTV model is useful for **market segmentation** and the allocation of marketing resources for __acquisition__, __retention__, and __cross selling__

# HOW
- total spending
- subcription length
- customer's age
- respond to disc. and offers

## customer value
$$ customer\_value = \frac{average\_order\_value * frequency\_of\_purchase} {customer\_churn} * profit\_margin $$

## average order value
$$ average\_order\_value = \frac{total\_revenue}{total\_orders} $$

## frequency of purchase
$$ frequency\_of\_purchase = \frac{total\_orders}{total\_customers} $$

* churn rate is the **%** of customers who have not ordered again
* customer lifetime = $ \frac{1}{churn\_rate} $
* repeat rate = 1 - churn_rate

# Objective(s)
1. identify most profitable customers
2. offer the best and make the most (high chance success to hit)
3. market segementation
4. budget estimation in order to acquire customers

In [ ]:
import pandas as pd

data = pd.read_csv('...')
data['revenue_each_order'] = data.Quantity * data.UnitPrice

# a. number of days between present date and the last purchase
# b. number of orders each customers
# c. sum of revenue_each_order

data_group = data.groupby('CustomerID').agg(
    num_days=('InvoiceDate', lambda date: (date.max() - date.min()).days),
    total_revenue=('revenue_each_order', sum),
    total_orders=('InvoiceNo', pd.Series.nunique),
)

## CLTV Calculation

### average_order_value
data_group['avg_order_value'] = data_group.total_revenue / data_group.total_orders

### frequency of purchase
data_group['freq_purchase'] = data_group.total_orders / data_group.shape[0]

### Repeat Rate / Churn Rate
### Churn Rate: customer do not come back (last_transaction == first_transaction)
### business need to explain what's the difference between stop (churn) and pause (dormant), by applying a certain threshold
churn_rate = data_group.select('num_days == 0').shape[0] / data_group.shape[0]# how many % left

### Profit Margin suppose that business making 25%
data_group['profit_margin'] = data_group.total_revenue * 0.25
### CLTV
data_group['CLTV'] = (data_group.avg_order_value * data_group.freq_purchase) / churn_rate * data_group.profit_margin

# Extract month and year from InvoiceDate
data['month_year'] = data['InvoiceDate'].apply(lambda x: x.strftime('%b-%Y'))

data.pivot_table(index=['CustomerID'], columns=['month_year'], values=['revenue_each_order'], aggfunc=sum, fill_value=0)

# Reference
- https://towardsdatascience.com/understanding-and-forecasting-customer-lifetime-value-cltv-634fe34f522b
- https://medium.com/swlh/predict-customer-lifetime-value-with-machine-learning-545624073d14
- https://towardsdatascience.com/measuring-users-with-customer-lifetime-value-cltv-94fccb4e532e

$$ \frac{\sum_{i, t}| y_{i, t} - \hat{y_{i, t}} |} {\sum_{i, t}} $$